# Data Preprocess for US Wheat Weather Data

### Necessary Imports

In [14]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
import sys
from fredapi import Fred

fred = Fred(api_key='2110034933792184d022f67f36bd7295')


In [37]:
STATE_STATION_MAP = {
    'ALABAMA': '72226',
    'ARIZONA': '72278',
    'ARKANSAS': '72340',
    'CALIFORNIA': '72390',
    'COLORADO': '72466',
    'CONNECTICUT': '72504',
    'DELAWARE': '72418',
    'FLORIDA': '120001',
    'GEORGIA': '130001',
    'IDAHO': '160001',
    'ILLINOIS': '170001',
    'INDIANA': '180001',
    'IOWA': '190001',
    'KANSAS': '200001',
    'KENTUCKY': '210001',
    'LOUISIANA': '220001',
    'MAINE': '230001',
    'MARYLAND': '240001',
    'MASSACHUSETTS': '250001',
    'MICHIGAN': '260001',
    'MINNESOTA': '270001',
    'MISSISSIPPI': '280001',
    'MISSOURI': '290001',
    'MONTANA': '300001',
    'NEBRASKA': '310001',
    'NEVADA': '320001',
    'NEW HAMPSHIRE': '330001',
    'NEW JERSEY': '340001',
    'NEW MEXICO': '350001',
    'NEW YORK': '360001',
    'NORTH CAROLINA': '370001',
    'NORTH DAKOTA': '380001',
    'OHIO': '390001',
    'OKLAHOMA': '400001',
    'OREGON': '410001',
    'PENNSYLVANIA': '420001',
    'RHODE ISLAND': '440001',
    'SOUTH CAROLINA': '450001',
    'SOUTH DAKOTA': '460001',
    'TENNESSEE': '470001',
    'TEXAS': '480001',
    'UTAH': '490001',
    'VERMONT': '500001',
    'VIRGINIA': '510001',
    'WASHINGTON': '530001',
    'WEST VIRGINIA': '540001',
    'WISCONSIN': '550001',
    'WYOMING': '560001',
}

STATE_CODE_MAP = {
    'ALABAMA': 'AL',
    'ARIZONA': 'AZ',
    'ARKANSAS': 'AR',
    'CALIFORNIA': 'CA',
    'COLORADO': 'CO',
    'CONNECTICUT': 'CT',
    'DELAWARE': 'DE',
    'FLORIDA': 'FL',
    'GEORGIA': 'GA',
    'IDAHO': 'ID',
    'ILLINOIS': 'IL',
    'INDIANA': 'IN',
    'IOWA': 'IA',
    'KANSAS': 'KS',
    'KENTUCKY': 'KY',
    'LOUISIANA': 'LA',
    'MAINE': 'ME',
    'MARYLAND': 'MD',
    'MASSACHUSETTS': 'MA',
    'MICHIGAN': 'MI',
    'MINNESOTA': 'MN',
    'MISSISSIPPI': 'MS',
    'MISSOURI': 'MO',
    'MONTANA': 'MT',
    'NEBRASKA': 'NE',
    'NEVADA': 'NV',
    'NEW HAMPSHIRE': 'NH',
    'NEW JERSEY': 'NJ',
    'NEW MEXICO': 'NM',
    'NEW YORK': 'NY',
    'NORTH CAROLINA': 'NC',
    'NORTH DAKOTA': 'ND',
    'OHIO': 'OH',
    'OKLAHOMA': 'OK',
    'OREGON': 'OR',
    'PENNSYLVANIA': 'PA',
    'RHODE ISLAND': 'RI',
    'SOUTH CAROLINA': 'SC',
    'SOUTH DAKOTA': 'SD',
    'TENNESSEE': 'TN',
    'TEXAS': 'TX',
    'UTAH': 'UT',
    'VERMONT': 'VT',
    'VIRGINIA': 'VA',
    'WASHINGTON': 'WA',
    'WEST VIRGINIA': 'WV',
    'WISCONSIN': 'WI',
    'WYOMING': 'WY'
}

### Load data

In [3]:
# ignore first row of the excel file
df = pd.read_excel("data/open-data-for-dot-density-map.xlsx", skiprows=1, dtype=str)

# remove the last row of the dataframe
df = df.iloc[:-1]

# print the last 5 rows of the dataframe
# # print the shape of the dataframe
# df.shape
# # print the columns of the dataframe
# df.columns
# # print the data types of the dataframe
# df.dtypes

### Get Coordinates from the state county codes

In [ ]:
# Print the first 5 rows of the dataframe
df.head()

In [ ]:
# Print the last 5 rows of the dataframe
df.tail()

### Create wheat-state proportion data

In [61]:
# 1. Filter class to all wheat
df_all_wheat = df[df['Class'] == 'All wheat'].copy()
# 2. Rename all the columns: State -> state, Class -> class, 2024 planted area (acres) -> planted_area
df_all_wheat.rename(columns={'State': 'state', 'Class': 'class', '2024 planted area (acres)': 'planted_area'}, inplace=True)

# 3. Uppercase the state column
df_all_wheat['state'] = df_all_wheat['state'].str.upper()
# 4. Make planted_area a float
df_all_wheat['planted_area'] = df_all_wheat['planted_area'].astype(float)
# 5. Group by State, aggregate the planted area
df_all_wheat_state = df_all_wheat.groupby('state').agg({'planted_area': 'sum'}).reset_index()
df_all_wheat_state['weight'] = df_all_wheat_state['planted_area'] / df_all_wheat_state['planted_area'].sum()
# print the last 5 rows of the dataframe
# print(df_all_wheat_state)
# 6. Save the dataframe to a csv file
df_all_wheat_state.to_csv('data/all_wheat_state_proportion.csv', index=False)
# print(df_all_wheat_state.tail())


### Get the station ID from each state

In [62]:
from meteostat import Point, Daily, Stations
from datetime import datetime
import pandas as pd


start = datetime(2000, 1, 1)
end = datetime(2025, 5, 1)

station_list = []

for state, sc in STATE_CODE_MAP.items():

    try:
        # Fetch all stations in US with coverage in the date range
        stations = Stations().region('US').inventory('daily', (start, end)).fetch()
        
        # Filter to just this state's stations
        stations_state = stations[stations['region'] == sc]

        if not stations_state.empty:
            # provide the first the station_id and station_name for the state
            # print all the station state and station_id in california
            print(stations_state[stations_state['region'] == 'CA'])
            station_id = stations_state.index[0]
            station_name = stations_state.loc[station_id, 'name']
            print(f"{state}: {station_id} - {station_name}")
            station_list.append({'state': state, 'station_id': station_id, 'station_name': station_name})
        else:
            print(f"{state}: No station with daily data in specified range.")
    except Exception as e:
        print(f"{state}: Error - {e}")

station_df = pd.DataFrame(station_list)
station_df.to_csv('data/station_df.csv', index=False)

Empty DataFrame
Columns: [name, country, region, wmo, icao, latitude, longitude, elevation, timezone, hourly_start, hourly_end, daily_start, daily_end, monthly_start, monthly_end]
Index: []
ALABAMA: 72223 - Mobile Regional Airport
Empty DataFrame
Columns: [name, country, region, wmo, icao, latitude, longitude, elevation, timezone, hourly_start, hourly_end, daily_start, daily_end, monthly_start, monthly_end]
Index: []
ARIZONA: 72274 - Tucson International Airport
Empty DataFrame
Columns: [name, country, region, wmo, icao, latitude, longitude, elevation, timezone, hourly_start, hourly_end, daily_start, daily_end, monthly_start, monthly_end]
Index: []
ARKANSAS: 72340 - North Little Rock, North Little Rock Airport
                                       name country region    wmo  icao  \
id                                                                        
72281         El Centro, Naval Air Facility      US     CA  72281  KNJK   
72286      Riverside / March Air Force Base      US    

### Aggregate weather data from 2000 to 2025

In [63]:
# Get the precipitation data from 2000 to 2025 for each state
# and based on the planted area, calculate weighted precipitation and tavg from each station, and aggregate them to combine to each date

# create a column with date, from 2000 to 2025
weather_records = []

for _, row in station_df.iterrows():
    state = row['state']
    station_id = row['station_id']
    weight = df_all_wheat_state.loc[df_all_wheat_state['state'] == state, 'weight'].values[0]

    data = Daily(station_id, start, end).fetch()
    if not data.empty:
        data = data[['tavg', 'prcp']].copy()
        data['date'] = data.index
        data['state'] = state
        data['weight'] = weight
        weather_records.append(data)


weather_df = pd.concat(weather_records)

daily_weighted = weather_df.groupby('date').apply(
    lambda g: pd.Series({
        'weighted_tavg': (g['tavg'].fillna(0) * g['weight']).sum(),
        'weighted_prcp': (g['prcp'].fillna(0) * g['weight']).sum()
    })
).reset_index()

# print the last 5 rows of the dataframe
# print(daily_weighted.tail())

# save the dataframe to a csv file
daily_weighted.to_csv('data/daily_weighted.csv', index=False)





In [15]:
print(daily_weighted.head())

NameError: name 'daily_weighted' is not defined

In [25]:
# import geopandas as gpd


# # Filter the original DataFrame to keep only "All wheat"
# df_filtered = df[df['class'] == 'All wheat'].copy()
# # 1. Load county shapefile from Census
# gdf = gpd.read_file("https://www2.census.gov/geo/tiger/GENZ2021/shp/cb_2021_us_county_5m.zip")

# # 2. Reproject to a projected CRS (EPSG:3857 for centroid accuracy)
# gdf_proj = gdf.to_crs(epsg=3857)

# # 3. Compute projected centroids
# gdf_proj['centroid_proj'] = gdf_proj.geometry.centroid

# # 4. Convert centroids back to lat/lon (EPSG:4326)
# gdf_proj = gdf_proj.set_geometry('centroid_proj').to_crs(epsg=4326)
# gdf_proj['centroid_lon'] = gdf_proj.geometry.x
# gdf_proj['centroid_lat'] = gdf_proj.geometry.y

# # 5. Merge with your DataFrame on state and county code
# df_with_coords = df_filtered.merge(
#     gdf_proj[['STATEFP', 'COUNTYFP', 'centroid_lat', 'centroid_lon']],
#     left_on=['State code', 'County code'],
#     right_on=['STATEFP', 'COUNTYFP'],
#     how='left'
# )

# df_with_coords = df_with_coords[['State code', 'County code', 'centroid_lat', 'centroid_lon', '2024 planted area (acres)']].drop_duplicates().reset_index(drop=True).sort_values(by=['State code', 'County code'])
# df_with_coords['2024 planted area (acres)'] = df_with_coords['2024 planted area (acres)'].astype(float)

# # weight of the area planted per county
# df_with_coords['weight'] = df_with_coords['2024 planted area (acres)'] / df_with_coords['2024 planted area (acres)'].sum()

# # 6. Final result
# print(df_with_coords)

# # save the dataframe to a csv file
# df_with_coords.to_csv('data/county_coords.csv', index=False)


In [ ]:
# example retrieval
from datetime import datetime, timedelta
from meteostat import Point, Daily

# Define location (latitude, longitude, elevation in meters)
location = Point(40.7128, -74.0060)  # Example: New York City

# Define date range
start = datetime(2020, 1, 1)
end = datetime(2020, 12, 31)

# Fetch daily data
data = Daily(location, start, end)
data = data.fetch()

print(data.head())

In [ ]:
# Define location (latitude, longitude, elevation in meters)
location = Point(40.7128, -74.0060)  # Example: New York City

# Define date range
start = datetime(2020, 1, 1)
end = datetime(2020, 12, 31)

# Fetch daily data
data = Daily(location, start, end)
data = data.fetch()

print(data.head())

In [24]:
# # retrieve data from meteostat
# import pandas as pd
# from meteostat import Point, Daily, Stations
# from datetime import datetime

# # Load your full dataset (replace with your actual CSV or DataFrame)
# # df = pd.read_csv("your_county_data.csv")  # or use your in-memory DataFrame

# # Filter out counties with zero planted area
# df_with_weight = df_with_coords[df_with_coords['2024 planted area (acres)'] > 0].copy()

# # Normalize weight
# # df_with_weight['weight'] = df_with_weight['2024 planted area (acres)'] / df_with_weight['2024 planted area (acres)'].sum()

# # Date range
# # start = datetime(2000, 1, 1)
# # start = datetime(2025, 6, 1)
# end = datetime.today() - timedelta(days=5)
# # past 20 years
# start = end - timedelta(days=7300)  # Past 30 days

# end = datetime.now()

# # Collect weather data
# weather_records = []

# for _, row in df_with_weight.iterrows():
#     try:
#         lat = row['centroid_lat']
#         lon = row['centroid_lon']
#         state = row['State code']
#         county = row['County code']

#         # Filter for stations with daily data in the desired date range
#         stations = Stations().nearby(lat, lon).fetch(10)  # Get top 5 nearby stations

#         if stations.empty:
#             print(f"No stations found for {state}-{county}")
#             continue

#         # Try each station until one returns valid weather data
#         weather = None
#         for station_id in stations.index:
#             data = Daily(station_id, start, end).fetch()
#             if not data.empty:
#                 weather = data
#                 break  # Stop when we get valid data

#         if weather is None:
#             print(f"No weather data available for any nearby station for {state}-{county}")
#             continue

#         # Process the weather data
#         weather = weather[['tavg', 'prcp']].copy()
#         weather['date'] = weather.index
#         weather['state_code'] = state
#         weather['county_code'] = county
#         weather['weight'] = row['weight']
#         weather_records.append(weather)

#     except Exception as e:
#         print(f"Error for {row['State code']}-{row['County code']}: {e}")

# # Combine all weather data
# weather_df = pd.concat(weather_records)

# # Compute weighted average per date
# daily_weighted = weather_df.groupby('date').apply(
#     lambda g: pd.Series({
#         'weighted_tavg': (g['tavg'] * g['weight']).sum(),
#         'weighted_prcp': (g['prcp'] * g['weight']).sum()
#     })
# ).reset_index()

# # Save or use as needed
# daily_weighted.to_csv("data/weighted_weather_2000_2025.csv", index=False)
# print(daily_weighted.head())

### Get data from yahoo finance (source = CME)

In [16]:
import yfinance as yf

# Get CME wheat data from 2000 to 2025
wheat_data = yf.download('ZW=F', start='2000-01-01', end='2025-05-30')

# print the last 5 rows of the dataframe
wheat_data.reset_index(inplace=True)
# print(wheat_data.tail())
# If MultiIndex columns exist, flatten them
if isinstance(wheat_data.columns, pd.MultiIndex):
    wheat_data.columns = wheat_data.columns.get_level_values(0)

# Reset index and rename for merging
# wheat_data = wheat_data.reset_index()[['Date', 'Close']]
# wheat_data = wheat_data.rename(columns={'Date': 'date', 'Close': 'wheat_price'})

# save the dataframe to a csv file
wheat_data.to_csv('data/wheat_data.csv', index=False)

/var/folders/yc/29rzgzxj7fvdl0qwf0ybn7_00000gn/T/ipykernel_37923/382012258.py:4: FutureWarning: YF.download() has changed argument auto_adjust default to True
  wheat_data = yf.download('ZW=F', start='2000-01-01', end='2025-05-30')
[*********************100%***********************]  1 of 1 completed


In [19]:
# merge weather data and wheat price data
wheat_data = pd.read_csv('data/wheat_data.csv')
# wheat_data['date'] = pd.to_datetime(wheat_data['date'])
wheat_data.rename(columns={'Date': 'date', 'Close': 'close', 'Open': 'open', 'High': 'high', 'Low': 'low', 'Volume': 'volume'}, inplace=True)

full_dates = pd.DataFrame({
    'date': pd.date_range(start='2000-01-01', end='2025-05-30')
})
wheat_data['date'] = pd.to_datetime(wheat_data['date'])
# drop the Date column
# wheat_data = wheat_data.drop(columns=['Date'])
wheat_data_full = pd.merge(full_dates, wheat_data, on='date', how='left')
wheat_data_full.tail()
wheat_data_full = wheat_data_full.ffill()
wheat_data_full.tail()


,date,close,high,low,open,volume
9277,2025-05-26,542.50,548.50,538.50,545.0,61665.0
9278,2025-05-27,528.50,544.00,527.50,543.5,66898.0
9279,2025-05-28,530.25,535.75,526.75,531.0,60152.0
9280,2025-05-29,534.00,534.75,527.25,533.0,48247.0
9281,2025-05-30,534.00,534.75,527.25,533.0,48247.0


In [ ]:

daily_weighted['date'] = pd.to_datetime(daily_weighted['date'])

# merge the two dataframes on the date column
# asof merge
# merged_data = pd.merge(wheat_data_full, daily_weighted, on='date', how='left')
merged_data = pd.merge_asof(wheat_data_full, daily_weighted, on='date', direction='backward')

# print the last 5 rows of the merged dataframe
# print(merged_data.head())

# save the merged dataframe to a csv file
merged_data.dropna(inplace=True)
merged_data.to_csv('data/data_for_model.csv', index=False)

NameError: name 'daily_weighted' is not defined

In [22]:
gdp = fred.get_series('GDP')
gdp.name = 'GDP'
print(gdp.tail())
fed_rate = fred.get_series('DFF')
fed_rate.name = 'Fed_Rate'
# print(fed_rate)
# print(fed_rate.loc['2010-01-01':'2020-12-31'])
# print(fed_rate.nunique())
# print(fed_rate.index.max())
cpi = fred.get_series('CPIAUCNS')
cpi.name = 'CPI'
# print(cpi.tail())

# # Convert to DataFrames with 'date' column
gdp = gdp.reset_index().rename(columns={'index': 'date'})
fed_rate = fed_rate.reset_index().rename(columns={'index': 'date'})
cpi = cpi.reset_index().rename(columns={'index': 'date'})

# # # merge the two dataframes on the date column
# merged_data = pd.merge(fed_rate, gdp, on='date', how='left')
# merged_data = pd.merge(merged_data, cpi, on='date', how='left')
# # ffill the merged data
# merged_data = merged_data.ffill()
# print(merged_data.tail())

# # save the merged dataframe to a csv file
# merged_data.to_csv('data/macro_data.csv', index=False)

# save the merged dataframe to a csv file
# merged_data.to_csv('data/merged_data.csv', index=False)

# NOTE: save file as macro_data.csv
merged_data = pd.merge_asof(fed_rate, gdp, on='date', direction='backward')
merged_data = pd.merge_asof(merged_data, cpi, on='date', direction='backward')
merged_data.to_csv('data/macro_data.csv', index=False)


2024-01-01    28624.069
2024-04-01    29016.714
2024-07-01    29374.914
2024-10-01    29723.864
2025-01-01    29962.047
Name: GDP, dtype: float64


In [23]:


# merge them asof backward one by one, start with daily, then fed rate, then cpi, then gdp
df_wheat = pd.read_csv('data/wheat_data.csv')

df_weather = pd.read_csv('data/daily_weighted.csv')

fed_rate = fred.get_series('DFF')
fed_rate.name = 'Fed_Rate'

cpi = fred.get_series('CPIAUCNS')
cpi.name = 'CPI'

gdp = fred.get_series('GDP')
gdp.name = 'GDP'

# Convert to DataFrames with 'date' column
gdp = gdp.reset_index().rename(columns={'index': 'date'})
fed_rate = fed_rate.reset_index().rename(columns={'index': 'date'})
cpi = cpi.reset_index().rename(columns={'index': 'date'})

# Rename columns
df_wheat.rename(columns={'Date': 'date', 'Close': 'close', 'Open': 'open', 'High': 'high', 'Low': 'low', 'Volume': 'volume'}, inplace=True)

# Convert to datetime
df_weather['date'] = pd.to_datetime(df_weather['date'])
df_wheat['date'] = pd.to_datetime(df_wheat['date'])
cpi['date'] = pd.to_datetime(cpi['date'])
fed_rate['date'] = pd.to_datetime(fed_rate['date'])
gdp['date'] = pd.to_datetime(gdp['date'])


df_merged = pd.merge_asof(df_wheat, df_weather, on='date', direction='backward')

df_merged = pd.merge_asof(df_merged, fed_rate, on='date', direction='backward')

df_merged = pd.merge_asof(df_merged, cpi, on='date', direction='backward')

df_merged = pd.merge_asof(df_merged, gdp, on='date', direction='backward')


# rename Date column from wheat_data to date
df_wheat.rename(columns={'Date': 'date'}, inplace=True)
# df_macro['date'] = pd.to_datetime(df_macro['date'])

df_merged.dropna(inplace=True)
# print the last 5 rows of the dataframe
print(df_merged.tail())
# save the merged dataframe to a csv file
df_merged.to_csv('data/merged_data.csv', index=False)

# Avoid division by zero and log of negative numbers
df_merged['high'] = np.maximum(df_merged['high'], df_merged['low'] + 1e-8)  # Ensure high > low
df_merged['open'] = np.maximum(df_merged['open'], 1e-8)  # Ensure open > 0
df_merged['close'] = np.maximum(df_merged['close'], 1e-8)  # Ensure close > 0

# Calculate log returns
log_hl = np.log(df_merged['high'] / df_merged['low'])
log_co = np.log(df_merged['close'] / df_merged['open'])
# Calculate realized volatility
df_merged['gk_vol_1d'] = np.sqrt(0.5 * log_hl**2 - (2 * np.log(2) - 1) * log_co**2)
df_merged['gk_vol_21d'] = df_merged['gk_vol_1d'].rolling(window=21).mean()
df_merged.dropna(inplace=True)

# print the last 5 rows of the dataframe
print(df_merged.tail())
# save the merged dataframe to a csv file
df_merged.to_csv('data/merged_data_with_realised_volatility.csv', index=False)


           date   close    high     low    open  volume  weighted_tavg  \
6228 2025-05-22  544.50  552.00  541.25  548.25   85214      15.694328   
6229 2025-05-23  542.50  548.50  538.50  545.00   61665      15.694328   
6230 2025-05-27  528.50  544.00  527.50  543.50   66898      15.694328   
6231 2025-05-28  530.25  535.75  526.75  531.00   60152      15.694328   
6232 2025-05-29  534.00  534.75  527.25  533.00   48247      15.694328   

      weighted_prcp  Fed_Rate      CPI        GDP  
6228       1.365375      4.33  321.465  29962.047  
6229       1.365375      4.33  321.465  29962.047  
6230       1.365375      4.33  321.465  29962.047  
6231       1.365375      4.33  321.465  29962.047  
6232       1.365375      4.33  321.465  29962.047  
           date   close    high     low    open  volume  weighted_tavg  \
6228 2025-05-22  544.50  552.00  541.25  548.25   85214      15.694328   
6229 2025-05-23  542.50  548.50  538.50  545.00   61665      15.694328   
6230 2025-05-27  528.

/Users/poyaohuang/Documents/Practice Program/Thesis/Agri_Forecast/venv/lib/python3.11/site-packages/pandas/core/arraylike.py:396: RuntimeWarning: invalid value encountered in sqrt
  result = getattr(ufunc, method)(*inputs, **kwargs)
